In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch.nn.modules.pooling import AdaptiveAvgPool2d
import timm
# from swin_transformer import SwinTransformer
from functools import partial

/home/schecter/miniconda3/envs/pikachu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = timm.create_model('swin_tiny_patch4_window7_224.ms_in1k', pretrained=True)
print("Model created")
print(model)

Model created
SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate='none')
       

In [3]:
in_features = 768
model.head = nn.Linear(in_features, in_features)

In [12]:
import cv2

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img.resize((224, 224, 3))
    img = torch.from_numpy(img).unsqueeze(0).float()
    img = torch.permute(img, (0, 3, 1, 2))
    return img

img = preprocess_image("../assets/annotated_image.jpg")

print(img.shape)

pred = (model(img))

print(pred.shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 7, 7, 768])


In [80]:
import swin_T_neck as neck
from importlib import reload
reload(neck)

<module 'swin_T_neck' from '/home/schecter/projects/small-object-detection/models/swin_T_neck.py'>

In [46]:
from swin_T_neck import swin_t

In [ ]:
net = swin_t()
pred2 = net(pred)
print(pred2.shape)

In [81]:
from swin_T_neck import UpMerging

In [82]:
um = UpMerging(768,7,2,2)

Input Size:  37632


In [79]:
pred = pred.permute(0, 3, 2, 1)
print(pred.shape)
print(um(pred).shape)

torch.Size([1, 768, 7, 7])


TypeError: rearrange() missing 1 required positional argument: 'pattern'